In [0]:
import dlt
from pyspark.sql.functions import (
    col, trim, lower, upper, when, regexp_replace, length
)
from pyspark.sql import Row

VOLUME_PATH = "/Volumes/workspace/damg7370_imdb/volume_final_project"


In [0]:
# ============================
# REFERENCE - REGION CODES
# ============================

@dlt.table(
    name="ref_region_codes",
    comment="Cleaned country/region codes (uppercased) for joining with IMDb region field",
    table_properties={
        "delta.columnMapping.mode": "name",
        "quality": "reference"
    }
)
def ref_region_codes():
    df_raw = (
        spark.read
            .option("header", "true")
            .csv(f"{VOLUME_PATH}/CountryCodes.csv")
    )

    df_clean = (
        df_raw
        # normalize names; adjust 'code' / 'country_name' if your headers differ
        .withColumn("region_code", upper(trim(col("Country"))))
        .withColumn("region_name", trim(col("codes")))

        # FIX common corrupted UTF-8 characters that came from Excel
        .withColumn("region_name",
        regexp_replace(col("region_name"), "�land Islands", "Åland Islands")
        )

        # drop header-like or empty rows
        .filter(col("region_code").isNotNull())
        .filter(col("region_code") != "")
        .filter(col("region_code") != "CODE")       # defensive if header leaked into data
    )

   # --- extra IMDB-specific historical region codes ---
    extra_regions = spark.createDataFrame(
        [
            ("CSHH", "Czechoslovakia"),
            ("CSXX", "Serbia and Montenegro"),
            ("DDDE", "German Democratic Republic (East Germany)"),
            ("SUHH", "Soviet Union (USSR)"),
            ("YUCS", "Yugoslavia"),
        ],
        ["region_code", "region_name"]
    )

    df_with_extra = (
        df_clean
        .select("region_code", "region_name")
        .unionByName(extra_regions)
        .dropDuplicates(["region_code"])
    )

    return df_with_extra


In [0]:
# ============================
# REFERENCE - LANGUAGE CODES
# ============================

@dlt.table(
    name="ref_language_codes",
    comment="Cleaned ISO language codes (Set1 2-letter, Set2/Set3 3-letter) for joining with IMDb language field",
    table_properties={
        "delta.columnMapping.mode": "name",
        "quality": "reference"
    }
)
def ref_language_codes():
    df_raw = (
        spark.read
            .option("header", "true")
            .csv(f"{VOLUME_PATH}/LanguageCodes.csv")
    )

    df_clean = (
        df_raw
        # normalize / rename
        .withColumn("language_name", trim(col("ISO Language Names")))
        .withColumn("code_set1", lower(trim(col("Set 1"))))   # usually 2-letter
        .withColumn("code_set2", lower(trim(col("Set 22"))))   # 3-letter variant
        .withColumn("code_set3", lower(trim(col("Set 23"))))   # 3-letter (often 639-3)

        # drop header row that sometimes leaks into data
        .filter(col("language_name") != "ISO Language Names")

        # keep only rows where we have at least one usable code
        .filter(
            (col("code_set1").isNotNull()) |
            (col("code_set2").isNotNull()) |
            (col("code_set3").isNotNull())
        )
        # Filter only ISO-valid rows
        .filter(
            (length(col("code_set1")) == 2) | (col("code_set1").isNull()) 
        )
        .filter(
            (length(col("code_set2")) == 3) | (col("code_set2").isNull())
        )
        .filter(
            (length(col("code_set3")) == 3) | (col("code_set3").isNull())
        )
    )

    # only keep the columns we actually need, deduplicated
    return (
        df_clean
        .select("language_name", "code_set1", "code_set2", "code_set3")
        .dropDuplicates(["language_name", "code_set1", "code_set2", "code_set3"])
    )